In [27]:
import random
import re
from collections import Counter
import numpy as np

Продвинутое машинное обучение: 
Домашнее задание 3
Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:
https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
расшифруйте их таким частотным методом.


In [28]:
CORPUS_FILE = 'AnnaKarenina.txt'

In [29]:
HIDDEN_TEXT = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [30]:
def make_encode_dict(symbols):
    """Generates random permutation dict based on list of symbols"""
    encoded = symbols.copy()
    random.shuffle(encoded)
    
    result = {}
    for o, e in zip(symbols, encoded):
        result[o] = e
    return result

def make_decode_dict(e_dict):
    """Makes decode dict based on encode dict"""
    result = {}
    for k, v in e_dict.items():
        result[v] = k
    return result

def encode_str(text, e_dict):
    """Encodes text with encode dict. For decode - just pass decode dict"""
    result = str()
    for c in text:
        result += e_dict[c]
    return result

In [31]:
def preprocess_text(text):
    return re.sub(r"\W+", " ", ''.join(text)).lower()

In [32]:
#Making corpus
with open(CORPUS_FILE, 'r') as f:
    corpus = f.readlines()
    
corpus = preprocess_text(corpus)

In [33]:
def get_freq_list(text):
    c = Counter(text)
    dl = len(text)
    result = []
    for k, v in c.items():
        result.append((k, v / dl))
    result = sorted(result, key=lambda x: x[1], reverse=True)
    return result

In [34]:
def find_decode_dict(corp_freq, text_freq, method='rank'):
    """Generates decode dictionary based on corpus freq & text freq.
    Has two methods - 'rank' - 1st to 1st, 2nd to 2nd etc, and 'closest' - finds closest freq. 
    Last method can map several tokens to one"""
    result = {}
    if method == 'rank':
        for i in range(min(len(corp_freq), len(text_freq))):
            result[text_freq[i][0]] = corp_freq[i][0]
    elif method == 'closest':
        for c, prob in text_freq:
            best_dist = 10
            best_char = None
            for cc, pp in corp_freq:
                if abs(pp - prob) < best_dist:
                    best_dist = abs(pp - prob)
                    best_char = cc
            result[c] = best_char
    return result

In [35]:
def measure_dict_quality(enc_dict, dec_dict):
    """Counts correct replacements"""
    counter = 0
    for a, b in enc_dict.items():
        try:
            if dec_dict[b] == a:
                counter += 1
        except:
            pass
    return counter / len(enc_dict)

In [36]:
def encode_decode(text, corpus, method='closest'):
    """Encodes text with random permutation and then decodes it using corpus"""
    text = preprocess_text(text)
    enc_dict = make_encode_dict(list(set(text)))
    enc_text = encode_str(text, enc_dict)
    corp_freq = get_freq_list(corpus)
    enc_freq = get_freq_list(enc_text)
    dec_dict = find_decode_dict(corp_freq, enc_freq, method)
    print(encode_str(enc_text, dec_dict))
    print(text)
    print('Dict quality: ', measure_dict_quality(enc_dict, dec_dict))

In [37]:
test_text1 = 'Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг'

In [38]:
test_text2 = preprocess_text('Быть энтузиасткой сделалось ее общественным положением, и иногда, когда ей даже того не хотелось, она, чтобы не обмануть ожиданий людей, знавших ее, делалась энтузиасткой. Сдержанная улыбка, игравшая постоянно на лице Анны Павловны, хотя и не шла к ее отжившим чертам, выражала, как у избалованных детей, постоянное сознание своего милого недостатка, от которого она не хочет, не может и не находит нужным исправляться.В середине разговора про политические действия Анна Павловна разгорячилась.— Ах, не говорите мне про Австрию! Я ничего не понимаю, может быть, но Австрия никогда не хотела и не хочет войны. Она предает нас. Россия одна должна быть спасительницей Европы. Наш благодетель знает свое высокое призвание и будет верен ему. Вот одно, во что я верю. Нашему доброму и чудному государю предстоит величайшая роль в мире, и он так добродетелен и хорош, что Бог не оставит его, и он исполнит свое призвание задавить гидру революции, которая теперь еще ужаснее в лице этого убийцы и злодея. Мы одни должны искупить кровь праведника На кого нам надеяться, я вас спрашиваю? Англия с своим коммерческим духом не поймет и не может понять всю высоту души императора Александра. Она отказалась очистить Мальту. Она хочет видеть, ищет заднюю мысль наших действий. Что они сказали Новосильцову? Ничего. Они не поняли, они не могут понять самоотвержения нашего императора, который ничего не хочет для себя и всё хочет для блага мира. И что они обещали? Ничего. И что обещали, и того не будет! Пруссия уж объявила, что Бонапарте непобедим и что вся Европа ничего не может против него И я не верю ни в одном слове ни Гарденбергу, ни Гаугвицу. Cette fameuse neutralité prussienne, ce nest quun piège. Этот пресловутый нейтралитет Пруссии — только западня. Я верю в одного Бога и в высокую судьбу нашего милого императора. Он спасет Европу! — Она вдруг остановилась с улыбкою насмешки над своею горячностью.')

In [39]:
encode_decode(test_text1, corpus)

влпу ии в лньуиьаоан баиубииось аолоп уш жалазоп лосзищлавло лоьвн баа нсии вр злоии в лобнсавн ансааврж уорррж щнирн лосслоьр ра и знлиал жаибс зри рн аол уш бласа басин зулвр щ лааалоп уньсавианиьра врунипнасп босааааь уоиьзн ар ороииьилавои ушн ларллнаррн сиаво и браоисп узоуоаь лолиби ари базии зр зраь п рн ьрою лол ьоблазлоббилавоаь аолаь ирауиаивррь ороииь аол баа уовоьан бласаа сунионб синуующиь иазибнслиь зоз
вряд ли в результате получилась такая уж хорошая расшифровка разве что если вы брали в качестве тестовых данных целые рассказы но и шерлок холмс был не так уж прост после буквы e которая действительно выделяется частотой дальше он анализировал уже конкретные слова и пытался угадать какими они могли бы быть я не знаю как запрограммировать такой интуитивный анализ так что давайте просто сделаем следующий логический шаг
Dict quality:  0.25


In [40]:
encode_decode(test_text2, corpus)

чзты цатпйнавтгож вкекаковы ее очэевтреаазп покойеанеп н наоука гоука еж кайе тоуо ае йотековы оаа чточз ае очпаапты ойнкаанж кйкеж йааршнй ее кекакавы цатпйнавтгож вкерйаааая пкзчга нураршая повтояаао аа кнэе аааз паркораз йотя н ае шка г ее отйнршнп чертап рзрайака гаг п нйчакорааазй кетеж повтояааое войааане вроеуо пнкоуо аековтатга от готороуо оаа ае йочет ае пойет н ае аайокнт апйазп нвпраркятывя р верекнае райуорора про покнтнчевгне кежвтрня аааа паркораа райуорячнкавы ай ае уорорнте пае про арвтрнй я анчеуо ае поанпай пойет чзты ао арвтрня ангоука ае йотека н ае йочет рожаз оаа прекает аав роввня окаа кокйаа чзты впавнтекыанэеж ерропз ааш чкауокетекы йаает врое рзвогое прнйраане н чпкет ререа епп рот окао ро что я рерй аашепп кочропп н чпкаопп уовпкарй преквтонт рекнчажшая рокы р пнре н оа таг кочрокетекеа н йорош что чоу ае овтарнт еуо н оа нвпокант врое прнйраане йакарнты ункрп рерокйэнн готорая теперы еэе пйаваее р кнэе цтоуо пчнжэз н йкокея пз окан кокйаз нвгппнты гроры прар

In [41]:
encode_decode(test_text2, corpus, method='rank')

гчть iнтпжиастыоз скерарось ее огцестленнчм уоройением и инодка ыодка ез кайе тодо не хотерось она бтогч не огманпть ойиканиз ршкез жналюих ее керарась iнтпжиастыоз скевйанная прчгыа идвалюая уостоянно на рище аннч уалролнч хотя и не юра ы ее отйилюим бевтам лчвайара ыаы п ижгароланнчх кетез уостоянное сожнание слоедо миродо некостатыа от ыотоводо она не хобет не мойет и не нахокит нпйнчм исувалряться л севекине важдолова уво уоритибесыие кезстлия анна уалролна важдовябирась ах не доловите мне уво алствиш я нибедо не уонимаш мойет гчть но алствия ниыодка не хотера и не хобет лознч она увекает нас воссия окна корйна гчть суаситерьнищез елвоуч наю градокетерь жнает слое лчсоыое увижлание и гпкет левен емп лот окно ло бто я левш наюемп когвомп и бпкномп доспкавш увекстоит лерибазюая ворь л миве и он таы когвокетерен и ховою бто год не осталит едо и он исуорнит слое увижлание жакалить диквп велоршщии ыотовая теуевь еце пйаснее л рище iтодо пгизщч и жрокея мч окни корйнч исыпуить ыволь увал

In [42]:
corp_freq = get_freq_list(corpus)
enc_freq = get_freq_list(HIDDEN_TEXT)
dec_dict = find_decode_dict(corp_freq, enc_freq, method='rank')
print(encode_str(HIDDEN_TEXT, dec_dict))

олна рд раяасо иевьтнуиды ана пемса иевьтнуиды соклс б шсеге леечжоиаз кесевды ногке пвемастсу лкевоо рлоге рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуиды чтнн йт пелнояиоо мосровсео йтятиао кбвлт хесз кеиомие з иамоге ио ечожтю


## Получается полный бред, но на больших текстах замена по рангам работает лучше, чем замена по вероятностям.

## 2 Биграммы
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
проведите тестирование аналогично п.1, но при помощи биграмм.


In [43]:
def get_ngram_freq_list(text, ngram=2, use_freq=True):
    result = []
    c = Counter()
    dl = len(text) - ngram + 1 #number of ngrams in text
    for i in range(dl):
        c.update([text[i:i + ngram]])
    if use_freq:
        for b, n in c.items():
            result.append((b, n / dl))
    else:
        for b, n in c.items():
            result.append((b, n))
    return sorted(result, key=lambda x: x[1], reverse=True)

def ngram_decode(text, dec_dict):
    result = []
    ngram = len(list(dec_dict.keys())[0])
    for i in range(0, len(text), ngram):
        try:
            b = dec_dict[text[i:i + ngram]]
            result.append(b)
        except:
            pass
    return ''.join(result)

In [44]:
#All together
enc_dict = make_encode_dict(list(set(test_text1)))
enc_text = encode_str(test_text1, enc_dict)
corp_freq = get_ngram_freq_list(corpus)
enc_freq = get_ngram_freq_list(enc_text)
dec_dict = find_decode_dict(corp_freq, enc_freq, method='rank')
print(ngram_decode(enc_text, dec_dict))
print(test_text1)

егасрее  елеобво соннаскмоавпота стоь  бмек с рагопоннэтнето о пх о денони н т чтвя е  етоет ивистнии лариовмит ыленсегопочтогоеманое трсоамаяскруалчадао  сй  бнаа  ихоеркао адис чди к ук ра дбе изнонвоич тсвенизсаь ек и убы ооввоужинняотя ем рнея кооч ква хактьо канеиде дн сшиь уддрроты к вжие вае кряте лоалесепийдао щеих отой ебелли пнь рнерота сн м рыыйазьнтьм отя ем о сй деноовл гдо ел ми оролтеопор зз вш лрелиутнитим ан
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг


### Получилось гораздо хуже - биграмм больше, статистика набирается хуже, особенно по тестовому куску. Если при посимвольной расшифровке всегда правильно угадывались пробелы, то сейчас и с ними проблема. Правда, я использовал совсем уж наивный метод построения расшифровки.

# 3 & 4 MCMC
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Расшифруйте сообщение:
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

In [45]:
def freq_list_to_dict(freq_list):
    result = {}
    for c, n in freq_list:
        result[c] = n
    return result

def get_dec_dict_score(text, decode_dict, corp_freq, ngram=2):
    score = 0
    decrypted_text = encode_str(text, decode_dict)
    decrypted_freq = freq_list_to_dict(get_ngram_freq_list(decrypted_text, ngram=ngram, use_freq=False))
    for k, v in decrypted_freq.items():
        if k in corp_freq:
            score += v * np.log(corp_freq[k])
    return score

def get_new_decode_dict(dec_dict, keep_spaces=False):
    """Randomly changes 2 keys. If keep_spaces - will not touch change them"""
    keys = None
    if keep_spaces:
        while keys == None:
            keys = random.sample(dec_dict.keys(), 2)
            if dec_dict[keys[0]] == ' ' or dec_dict[keys[1]] == ' ':
                keys = None
    else:
        keys = random.sample(dec_dict.keys(), 2)
    new_dict = dec_dict.copy()
#     print(keys)
    new_dict[keys[0]], new_dict[keys[1]] = new_dict[keys[1]], new_dict[keys[0]]
    return new_dict

def random_coin(p):
    if random.random() > p:
        return False
    else:
        return True
    
def init_dict(text, corpus):
    """Initial dict for MCMC based on top frequencies. Same as char decoding"""
    result = {}
    text_freq = get_freq_list(text)
    corp_freq = get_freq_list(corpus)
    for i in range(len(text_freq)):
        result[text_freq[i][0]] = corp_freq[i][0]
    for i in range(len(text_freq), 34):
        result['_' * i] = corp_freq[i][0]
    return result

In [110]:
def mcmc_decode(text, corp_freq, n_iter=10000, corpus=corpus, ngram=2, print_progress=True):
    cur_decode_dict = init_dict(text, corpus)
    best_dict = None
    score = -1e+40
    for i in range(n_iter):
        new_decode_dict = get_new_decode_dict(cur_decode_dict, keep_spaces=True)
        cur_score = get_dec_dict_score(text, cur_decode_dict, corp_freq, ngram=ngram)
        new_score = get_dec_dict_score(text, new_decode_dict, corp_freq, ngram=ngram)
        acceptance_prob = min(1, np.exp(new_score - cur_score))
#         print(cur_score, new_score)
        if cur_score > score:
            score = cur_score
            best_dict = cur_decode_dict
        if random_coin(acceptance_prob):
            cur_decode_dict = new_decode_dict
        if (i + 1) % 500 == 0 and print_progress:
            print(i, ':', encode_str(text, best_dict))
            print(score)
    return best_dict

In [114]:
corp_freq = get_ngram_freq_list(corpus, ngram=2, use_freq=False)
corp_freq_dict = freq_list_to_dict(corp_freq)
best_dict = mcmc_decode(HIDDEN_TEXT, corp_freq_dict, 20000)

499 : орна бы бакасо тевшинятым ана дезса тевшинятым согрс у жселе реепщотаю гесевым нолге двезасися ргевоо броле бы бро рконина двибаняте а денузасо шиграшинятым пинн чи дерноктоо зосбовсео чикитао гуври цесю гетозте ю тазоле то епощих
1918.7360678593677
999 : арни бы бижиса товшенятым ини кодси товшенятым сапрс у юсоло роогцатих посовым налпо кводисеся рповаа брало бы бра ржанени квебинято и конудиса шепришенятым генн че корнажтаа дасбавсоа чежетиа пувре зосх потадто х тидало та огацей
1938.4059222526482
1499 : ерна бы бамале товшинятых ана досла товшинятых лепрл у флого роожцетак половых негпо двосалиля рповее брего бы бре рменина двибанято а донусале шипрашинятых жинн чи дорнемтее селбевлое чимитае пуври золк потесто к тасего те ожеций
1969.8225871087022
1999 : ерна бы бамале товшинятый ана содла товшинятый лепрл у злого роожчетак половый негпо сводалиля рповее брего бы бре рменина свибанято а сонудале шипрашинятый жинн фи сорнемтее делбевлое фимитае пуври холк потедто к тадего те 

### Получилось вполне пристойно. Перепутаны буквы М и Д, проблема с Ю - но она одна в нашем тексте и идет последней. С учетом короткости выборки - нормальный результат. Тем не менее, у функции много локальных максмумов и часто она застревает совсем не там. 



# 5 Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [116]:
for i in range(2, 11):
    corp_freq = get_ngram_freq_list(corpus, ngram=i, use_freq=False)
    corp_freq_dict = freq_list_to_dict(corp_freq)
    best_dict = mcmc_decode(HIDDEN_TEXT, corp_freq_dict, 20000, ngram=i, print_progress=False)
    print('For ngram = ', i)
    print(encode_str(HIDDEN_TEXT, best_dict))

For ngram =  2
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
For ngram =  3
если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю
For ngram =  4
если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю
For ngram =  5
ирла бы багами нескольный ала вечма нескольный мижрм у эмете реедхиная жемесый литже всечамомь ржесии брите бы бри ргилола всобальне а велучами кожракольный долл по верлигнии чимбисмеи погонаи жусро земя женичне я начите ни едихоe
For ngram =  6
и

### По ощущениям, 3-5 граммы работают лучше и стабильнее биграмм - реже попадают в неправильные максимумы, разбираются с редкими и близкими буквами (см. М - Д и Ю в конце). При увеличении размера все ломается из-за недостаточности статистики. При размере 6 еще можно получить нормальный результат, при большей длине мне ни разу не удалось.